# Problem Complexity
Menjadwalkan : 
- m = 1 mahasiswa (kemunginan : 124)
- d = 1 atau 2 dosen untuk mendampingi (kemungkinan : 3)
- s = pada 1 slot tempat x waktu (kemngkinan: 13 * 20)

Complexity : $$ C_{m}^s \times d$$

Order of Complexity : Combinatorial

In [18]:
comb((13*20), 124) * 3

208501360578020938029361385623653699574688684968360947127928832021966924612000

In [10]:
import pandas as pd
import numpy as np
from natsort import natsorted

In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
mhs = pd.read_csv('data/mahasiswa.csv')
dosen = pd.read_csv('data/dosen.csv')
ruang = pd.read_csv('data/ruang.csv')
jadwal_dosen = pd.read_csv('data/jadwal_dosen.csv')

In [6]:
jdw = jadwal_dosen.pivot_table(
    index='dosen', 
    columns=['day', 'time'], 
    values='val',
    aggfunc='sum'
)[['Senin', 'Selasa', 'Rabu', 'Kamis', 'Jumat']]

In [7]:
jdw

day      Senin                   Selasa                    Rabu              \
time     08:00 09:30 13:30 15:30  08:00 09:30 13:30 15:30 08:00 09:30 13:30   
dosen                                                                         
dosen_0    0.0   0.0   1.0   1.0    0.0   0.0   1.0   0.0   0.0   0.0   1.0   
dosen_1    0.0   1.0   0.0   1.0    0.0   1.0   1.0   0.0   0.0   1.0   1.0   
dosen_10   0.0   0.0   0.0   0.0    0.0   0.0   0.0   1.0   0.0   1.0   0.0   
dosen_11   0.0   1.0   0.0   0.0    0.0   1.0   1.0   1.0   0.0   1.0   0.0   
dosen_12   0.0   0.0   0.0   0.0    0.0   1.0   0.0   0.0   0.0   0.0   0.0   
...        ...   ...   ...   ...    ...   ...   ...   ...   ...   ...   ...   
dosen_76   0.0   0.0   0.0   0.0    0.0   1.0   1.0   1.0   0.0   0.0   1.0   
dosen_77   0.0   1.0   1.0   1.0    0.0   0.0   0.0   1.0   0.0   0.0   0.0   
dosen_78   0.0   0.0   0.0   0.0    0.0   0.0   0.0   1.0   0.0   1.0   1.0   
dosen_8    0.0   1.0   1.0   1.0    0.0   1.0   0.0   0.0   0.0   0.0   0.0   
dosen_9    0.0   1.0   1.0   1.0    0.0   0.0   0.0   0.0   0.0   1.0   1.0   

day            Kamis                   Jumat                    
time     15:30 08:00 09:30 13:30 15:30 08:00 09:30 13:30 15:30  
dosen                                                           
dosen_0    0.0   0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0  
dosen_1    0.0   0.0   1.0   1.0   0.0   0.0   0.0   0.0   1.0  
dosen_10   1.0   0.0   0.0   0.0   1.0   0.0   1.0   0.0   1.0  
dosen_11   0.0   0.0   1.0   0.0   0.0   0.0   1.0   0.0   0.0  
dosen_12   1.0   0.0   0.0   1.0   1.0   0.0   0.0   1.0   1.0  
...        ...   ...   ...   ...   ...   ...   ...   ...   ...  
dosen_76   1.0   0.0   0.0   1.0   1.0   0.0   1.0   0.0   1.0  
dosen_77   0.0   0.0   1.0   0.0   1.0   0.0   0.0   1.0   1.0  
dosen_78   1.0   0.0   1.0   0.0   1.0   0.0   1.0   1.0   0.0  
dosen_8    0.0   0.0   0.0   1.0   1.0   0.0   1.0   0.0   1.0  
dosen_9    1.0   0.0   1.0   0.0   1.0   0.0   0.0   0.0   1.0  

[79 rows x 20 columns]

# State Definition 

In [8]:
state = mhs[['nama']]
state['dp1'] = 0
state['dp2'] = 0 
state['ruang'] = np.random.choice(ruang.ruang, size=len(mhs))
state['time'] = np.random.choice(jadwal_dosen['time'].unique(),  size=len(mhs))

In [9]:
state

,nama,dp1,dp2,ruang,time
0,mhs_0,0,0,Lab Komputasi,13:30
1,mhs_1,0,0,Lab Basis Data,09:30
2,mhs_2,0,0,Lab UC,09:30
3,mhs_3,0,0,Lab Foresty,08:00
4,mhs_4,0,0,Ruang Rapat ICM,09:30
...,...,...,...,...,...
119,mhs_119,0,0,Lab Foresty,13:30
120,mhs_120,0,0,Lab UC,09:30
121,mhs_121,0,0,Lab Basis Data,15:30
122,mhs_122,0,0,Lab HES,13:30


# Fit/Objective Function 
Dengan:
- Jumlah dosen yang hadir (paling severe) = a
- Jumlah bentrok jadwal dosen = b
- Kesesuaian kk ruang dan dosen = c 
- durasi jadwal = d

Minimalkan nilai e :

$$ e = \frac{\beta \cdot b + \delta \cdot d}{\alpha \cdot a+ \gamma \cdot c}$$

with 
- $\alpha = 2$
- $\beta = 1.5$
- $\gamma = 1$
- $\delta = 5$

best case 
a = 2*100
b = 0 
c = 100
